In [4]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import copy

ROOT_DIR = os.path.abspath("../../")
DIR = os.path.join(ROOT_DIR, "datasets/treering/val")

In [3]:
vec = list(range(6))
vec

[0, 1, 2, 3, 4, 5]

In [6]:
import json
import numpy as np
import copy
 
def getRegion(imageV,type = 'Ring'):
    goodRegions = []
    orderX = []
    it = iter(imageV)
    for region in it:
        regAtt = region['region_attributes']
        shAtt = region['shape_attributes']
        if shAtt['name'] == 'polyline':
            for regk, regv in regAtt.items():
                if regk == type or regv == type:
                    goodRegions.append(region)
                    orderX.append(np.average(shAtt['all_points_x']))
    if not goodRegions:
        print("No data")
        return None
    return {'goodRegions':goodRegions, 'orderX':(np.argsort(orderX))}
 
newJ = {}
shpAttrBase = {'name':'polygon'}
with open(os.path.join(DIR, 'via_region_data.json'),'r') as json_file:
    dct = json.load(json_file)
    for image, imageAtt in dct.items():
        print("Which image:",' ',image)
        newJ[image] = {}
        for imageK, imageV in imageAtt.items():
            print("imageKeys:",' ',imageK)
            if imageK == 'regions':
                print("unsortedRegions:",' ',imageV)
                sortRegions = getRegion(imageV,type='Ring')
                if sortRegions == None:
                    print("continue, no sortRegions")
                    continue;
                print("sortRegions:",' ',sortRegions)
                print('orderX: ',sortRegions['orderX'])
                shpAtt = shpAttrBase.copy()
                newJ[image]['regions'] = []
                for i in range(len(sortRegions['orderX'])-1):
                    print(i)
                    firstRegion = sortRegions['goodRegions'][sortRegions['orderX'][i]]['shape_attributes']
                    secondRegion = sortRegions['goodRegions'][sortRegions['orderX'][i+1]]['shape_attributes']
                    print('firstRegion: ',firstRegion)
                    print('secondRegion: ',secondRegion)
                    shpAtt['all_points_x'] = firstRegion['all_points_x'] + secondRegion['all_points_x'][::-1]
                    shpAtt['all_points_y'] = firstRegion['all_points_y'] + secondRegion['all_points_y'][::-1]
                    print('newAtt: ',shpAtt)
                    regions = {'shape_attributes':shpAtt,'region_attributes':{'type':'RingBndy'}}
                    newJ[image]['regions'].append(copy.deepcopy(regions))
            else:
                #print(imageV)
                try:
                    newJ[image][imageK] = imageV
                except:
                    print("can't append:",' ',imageK,",",imageV)
        with open(os.path.join(DIR,'via_region_data_transform.json'),'w') as outfile:
            json.dump(newJ, outfile,indent=4)

Which image:   RSA1A_8-Image Export-09.jpg1975852
imageKeys:   filename
imageKeys:   size
imageKeys:   regions
unsortedRegions:   [{'shape_attributes': {'name': 'polyline', 'all_points_x': [428, 416, 427, 430, 427, 420, 439, 434, 447, 444, 455, 452, 451, 458, 453, 453, 434, 437, 430, 420, 418, 427, 435, 429, 435, 425, 428, 423, 433, 439, 427, 434, 434, 442, 446, 444, 439], 'all_points_y': [10, 47, 101, 161, 172, 175, 281, 323, 418, 490, 603, 656, 660, 673, 697, 761, 886, 923, 967, 1040, 1064, 1091, 1115, 1206, 1261, 1300, 1341, 1452, 1515, 1545, 1603, 1632, 1672, 1746, 1794, 1851, 1876]}, 'region_attributes': {'Ring': ''}}, {'shape_attributes': {'name': 'polyline', 'all_points_x': [1439, 1425, 1421, 1427, 1415, 1419, 1411, 1421, 1424, 1427, 1423, 1421, 1412, 1407, 1410, 1396, 1400, 1393, 1392, 1389, 1384, 1373, 1386, 1396, 1396, 1393, 1389], 'all_points_y': [34, 130, 197, 273, 361, 448, 521, 572, 623, 678, 793, 882, 983, 1004, 1065, 1181, 1281, 1324, 1382, 1455, 1580, 1622, 1674, 1719,